# Maximum Likelihood Estimation: The Normal Linear Model

The following tutorial will introduce maximum likelihood estimation
in Julia for the normal linear model.

The normal linear model (sometimes referred to as the OLS model) is
the workhorse of regression modeling and is utilized across a number
of diverse fields.  In this tutorial, we will utilize simulated data
to demonstrate how Julia can be used to recover the parameters of
interest.

The first order of business is to use the `Optim` package
and also include the `NLSolversBase` routine:

In [1]:
using Optim, NLSolversBase, Random
using LinearAlgebra: diag
Random.seed!(0);                            # Fix random seed generator for reproducibility

The first item that needs to be addressed is the data generating process or DGP.
The following code will produce data from a normal linear model:

In [2]:
n = 500                             # Number of observations
nvar = 2                            # Number of variables
β = ones(nvar) * 3.0                # True coefficients
x = [ones(n) randn(n, nvar - 1)]    # X matrix of explanatory variables plus constant
ε = randn(n) * 0.5                  # Error variance
y = x * β + ε;                      # Generate Data

In the above example, we have 500 observations, 2 explanatory
variables plus an intercept, an error variance equal to 0.5,
coefficients equal to 3.0, and all of these are subject to change by
the user. Since we know the true value of these parameters, we
should obtain these values when we maximize the likelihood function.

The next step in our tutorial is to define a Julia function for the
likelihood function. The following function defines the likelihood
function for the normal linear model:

In [3]:
function Log_Likelihood(X, Y, β, log_σ)
    σ = exp(log_σ)
    llike = -n/2*log(2π) - n/2* log(σ^2) - (sum((Y - X * β).^2) / (2σ^2))
    llike = -llike
end

Log_Likelihood (generic function with 1 method)

The log likelihood function accepts 4 inputs: the matrix of
explanatory variables (X), the dependent variable (Y), the β's, and
the error varicance. Note that we exponentiate the error variance in
the second line of the code because the error variance cannot be
negative and we want to avoid this situation when maximizing the
likelihood.

The next step in our tutorial is to optimize our function. We first
use the `TwiceDifferentiable` command in order to obtain the Hessian
matrix later on, which will be used to help form t-statistics:

In [4]:
func = TwiceDifferentiable(vars -> Log_Likelihood(x, y, vars[1:nvar], vars[nvar + 1]),
                           ones(nvar+1); autodiff=:forward);

The above statment accepts 4 inputs: the x matrix, the dependent
variable y, and a vector of β's and the error variance.  The
`vars[1:nvar]` is how we pass the vector of β's and the `vars[nvar +
1]` is how we pass the error variance. You can think of this as a
vector of parameters with the first 2 being β's and the last one is
the error variance.

The `ones(nvar+1)` are the starting values for the parameters and
the `autodiff=:forward` command performs forward mode automatic
differentiation.

The actual optimization of the likelihood function is accomplished
with the following command:

In [5]:
opt = optimize(func, ones(nvar+1))

 * Status: success

 * Candidate solution
    Minimizer: [3.00e+00, 2.96e+00, -6.49e-01]
    Minimum:   3.851229e+02

 * Found with
    Algorithm:     Newton's Method
    Initial Point: [1.00e+00, 1.00e+00, 1.00e+00]

 * Convergence measures
    |x - x'|               = 2.62e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.72e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 5.12e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.33e-15 ≰ 0.0e+00
    |g(x)|                 = 3.41e-13 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    7
    f(x) calls:    31
    ∇f(x) calls:   31
    ∇²f(x) calls:  7


The first input to the command is the function we wish to optimize
and the second input are the starting values.

After a brief period of time, you should see output of the
optimization routine, with the parameter  estimates being very close
to our simulated values.

The optimization routine stores several quantities and we can obtain
the maximim likelihood estimates with the following command:

In [6]:
parameters = Optim.minimizer(opt)

3-element Array{Float64,1}:
  3.002788633849947
  2.964549617572727
 -0.648692780562844

!!! Note
    Fieldnames for all of the quantities can be obtained with the following command:
    fieldnames(opt)

Since we paramaterized our likelihood to use the exponentiated
value, we need to exponentiate it to get back to our original log
scale:

In [7]:
parameters[nvar+1] = exp(parameters[nvar+1])

0.5227286513837306

In order to obtain the correct Hessian matrix, we have to "push" the
actual parameter values that maximizes the likelihood function since
the `TwiceDifferentiable` command uses the next to last values to
calculate the Hessian:

In [8]:
numerical_hessian = hessian!(func,parameters)

3×3 Array{Float64,2}:
 175.766        -12.0877        5.67464e-14
 -12.0877       182.437         5.88344e-15
   5.67464e-14    5.88344e-15  96.0542     

We can now invert our Hessian matrix to obtain the variance-covariance matrix:

In [9]:
var_cov_matrix = inv(numerical_hessian)

3×3 Array{Float64,2}:
  0.00571544    0.000378687  -3.39973e-18
  0.000378687   0.00550643   -5.60994e-19
 -3.39973e-18  -5.60994e-19   0.0104108  

In this example, we are only interested in the statistical
significance of the coefficient estimates so we obtain those with
the following command:

In [10]:
β = parameters[1:nvar]

2-element Array{Float64,1}:
 3.002788633849947
 2.964549617572727

We now need to obtain those elements of the variance-covariance
matrix needed to obtain our t-statistics, and we can do this with
the following commands:

In [11]:
temp = diag(var_cov_matrix)
temp1 = temp[1:nvar]

2-element Array{Float64,1}:
 0.005715441191953887
 0.005506430533329349

The t-statistics are formed by dividing element-by-element the
coefficients by their standard errors, or the square root of the
diagonal elements of the variance-covariance matrix:

In [12]:
t_stats = β./sqrt.(temp1)

2-element Array{Float64,1}:
 39.719144251161474
 39.95063081460274 

From here, one may examine other statistics of interest using the
output from the optimization routine.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*